<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Data_Analysis_Machine_Learning/blob/master/1.%20Business%20Intelligence%20and%20Sales%20Analysis/Online%20Retail%20Sales%20(Clustering)/Online_Retail_Sales_Cluestering_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Content
* 1. [Preparing Project](#1)
    * 1.1 [Import Libraries](#1.1)
    * 1.2 [Import Dataset](#1.2)
* 2. [Dataset Analysis](#2)
    * 2.1 [Understanding Dataset's Variable](#2.1)
    * 2.2 [Statistical Analysis](#2.2)
    


## 1. Preparing Project <a id=1></a>

### 1.1 Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1.2 Import Dataset <a id=-1.2></a>

In [28]:
df = pd.read_excel("/content/Online Retail.xlsx")

In [29]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## 2. Dataset Analysis <a id=2></a>

In [30]:
df.shape

(541909, 8)

In [31]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

### 2.1 Understaning Dataset's Variables <a id=2.1></a>

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


1. **InvoiceNo**: A unique identifier for each transaction (invoice) in the dataset. It is an object (string) data type.
2. **StockCode**: A unique identifier for each product (item) being sold. It is an object (string) data type.
3. **Description**: A brief description of the product. It is an object (string) data type. Some descriptions are missing (540,455 non-null entries).
4. **Quantity**: The quantity of the product purchased in each transaction. It is an integer (int64) data type.
5. **InvoiceDate**: The date and time when the transaction (invoice) was generated. It is a datetime (datetime64[ns]) data type.
6. **UnitPrice**: The price per unit of the product. It is a floating-point (float64) data type.
7. **CustomerID**: A unique identifier for each customer. It is a floating-point (float64) data type. Some customer IDs are missing (406,829 non-null entries).
8. **Country**: The country where the customer resides. It is an object (string) data type.

### 2.2 Statistical Analysis <a id=2.2></a>

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


#### Quantity
- **Count**: 541,909 entries.
- **Mean**: Average quantity purchased per transaction is approximately 9.55 units.
- **Min**: The minimum quantity is -80,995, indicating possible returns or data entry errors.
- **25%**: 25th percentile is 1, meaning 25% of transactions involve purchasing only 1 unit.
- **50% (Median)**: The median quantity is 3 units.
- **75%**: 75th percentile is 10 units, meaning 75% of transactions involve purchasing up to 10 units.
- **Max**: The maximum quantity is 80,995 units, suggesting either a large bulk purchase or potential data entry error.
- **Standard Deviation (std)**: The standard deviation is 218.08, indicating high variability in the quantity of items purchased.

#### InvoiceDate
- **Count**: 541,909 entries.
- **Mean**: The average date of transactions is around July 4, 2011.
- **Min**: The earliest transaction date is December 1, 2010.
- **25%**: The first quartile date is March 28, 2011.
- **50% (Median)**: The median transaction date is July 19, 2011.
- **75%**: The third quartile date is October 19, 2011.
- **Max**: The latest transaction date is December 9, 2011.

#### UnitPrice
- **Count**: 541,909 entries.
- **Mean**: The average unit price is approximately $4.61.
- **Min**: The minimum price is -$11,062.06, indicating possible data errors (negative prices are invalid).
- **25%**: The 25th percentile price is $1.25.
- **50% (Median)**: The median unit price is $2.08.
- **75%**: The 75th percentile price is $4.13.
- **Max**: The maximum price is $38,970, suggesting outliers or high-priced items.
- **Standard Deviation (std)**: The standard deviation is 96.76, showing high variability in unit prices.

#### CustomerID
- **Count**: 406,829 entries (missing values for some transactions).
- **Mean**: The average customer ID is approximately 15,287.69.
- **Min**: The minimum customer ID is 12,346.
- **25%**: The 25th percentile customer ID is 13,953.
- **50% (Median)**: The median customer ID is 15,152.
- **75%**: The 75th percentile customer ID is 16,791.
- **Max**: The maximum customer ID is 18,287.
- **Standard Deviation (std)**: The standard deviation is 1,713.60, indicating a wide range of customer IDs.